In [1]:
import pandas as pd

In [2]:
csv_filename = './../cleaned_database/movies.csv'
df = pd.read_csv(csv_filename)

C:\Users\RohanDalvi\AppData\Local\Temp\ipykernel_1076\348238252.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_filename)


In [3]:
def parse_list(cell):
    if pd.isnull(cell) or cell == "":
        return []
    return [item.strip() for item in cell.split(", ")]

In [4]:
movie_data_dict = {}

for _, row in df.iterrows():
    movie_id = row["Movie ID"]
    movie_data_dict[movie_id] = {
        "name": row["Name"],
        "date": row["Date"],
        "tagline": row["Tagline"],
        "description": row["Description"],
        "minute": row["Duration"],  # 'Duration' was originally stored as 'minute'
        "rating": row["Rating"],
        "actors": parse_list(row["Actors"]),
        "roles": parse_list(row["Roles"]),
        "crew": parse_list(row["Crew"]),
        "work": parse_list(row["Work"]),
        "countries": parse_list(row["Countries"]),
        "genres": parse_list(row["Genres"]),
        "languages": parse_list(row["Languages"]),
        "types": parse_list(row["Types"]),
        "posters": parse_list(row["Posters"]),
        "studios": parse_list(row["Studios"]),
        "themes": parse_list(row["Themes"])
    }

In [5]:
first_5_keys = list(movie_data_dict.keys())[:5]  
for key in first_5_keys:
    print(f"Movie ID: {key}, Data: {movie_data_dict[key]}")


Movie ID: 1000001, Data: {'name': 'Barbie', 'date': 2023.0, 'tagline': "She's everything. He's just Ken.", 'description': 'Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of living among humans.', 'minute': 114.0, 'rating': 3.86, 'actors': ['Margot Robbie', 'Ryan Gosling', 'America Ferrera', 'Ariana Greenblatt', 'Issa Rae', 'Kate McKinnon', 'Alexandra Shipp', 'Emma Mackey', 'Hari Nef', 'Sharon Rooney', 'Ana Cruz Kayne', 'Ritu Arya', 'Dua Lipa', 'Nicola Coughlan', 'Emerald Fennell', 'Simu Liu', 'Kingsley Ben-Adir', 'Ncuti Gatwa', 'Scott Evans', 'John Cena', 'Michael Cera', 'Rhea Perlman', 'Helen Mirren', 'Will Ferrell', 'Connor Swindells', 'Jamie Demetriou', 'Andrew Leung', 'Will Merrick', 'Zheng Xi Yong', 'Asim Chaudhry', 'Ray Fearon', 'Erica Ford', 'Hannah Khalique-Brown', 'Mette', 'Marisa Abela', 'Lucy Boynton', 'Rob Brydon', 'T

In [6]:
!pip install chromadb

In [7]:
import chromadb
chroma_client = chromadb.PersistentClient(path="./../chromadb_client")


In [8]:
chroma_client.delete_collection(name="movies_database")

ValueError: Collection movies_database does not exist.

In [9]:
collection = chroma_client.create_collection(name="movies_database")


In [10]:
def insert_movies_to_chroma(movie_data_dict):
    documents = []
    metadatas = []
    ids = []
    separator = ","


    for movie_id, movie_data in movie_data_dict.items():
        document = f"{movie_data['name']} - {movie_data['description']}"
        metadata = {
            "date": movie_data.get("date"),
            "tagline": movie_data.get("tagline"),
            "minute": movie_data.get("minute"),
            "rating": movie_data.get("rating"),
            "actors": separator.join(movie_data.get("actors", [])),
            "roles": separator.join(movie_data.get("roles", [])),
            "crew": separator.join(movie_data.get("crew", [])),
            "work": separator.join(movie_data.get("work", [])),
            "countries": separator.join(movie_data.get("countries", [])),
            "genres":separator.join( movie_data.get("genres", [])),
            "languages": separator.join(movie_data.get("languages", [])),
            "types": separator.join(movie_data.get("types", [])),
            "posters": separator.join(movie_data.get("posters", [])),
            "studios": separator.join(movie_data.get("studios", [])),
            "themes": separator.join(movie_data.get("themes", []))
        }
        documents.append(document)
        metadatas.append(metadata)
        ids.append(str(movie_id))

    collection.add(documents=documents, metadatas=metadatas, ids=ids)


In [25]:
print(movie_data_dict[1000001])

{'name': 'Barbie', 'date': 2023.0, 'tagline': "She's everything. He's just Ken.", 'description': 'Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of living among humans.', 'minute': 114.0, 'rating': 3.86, 'actors': ['Margot Robbie', 'Ryan Gosling', 'America Ferrera', 'Ariana Greenblatt', 'Issa Rae', 'Kate McKinnon', 'Alexandra Shipp', 'Emma Mackey', 'Hari Nef', 'Sharon Rooney', 'Ana Cruz Kayne', 'Ritu Arya', 'Dua Lipa', 'Nicola Coughlan', 'Emerald Fennell', 'Simu Liu', 'Kingsley Ben-Adir', 'Ncuti Gatwa', 'Scott Evans', 'John Cena', 'Michael Cera', 'Rhea Perlman', 'Helen Mirren', 'Will Ferrell', 'Connor Swindells', 'Jamie Demetriou', 'Andrew Leung', 'Will Merrick', 'Zheng Xi Yong', 'Asim Chaudhry', 'Ray Fearon', 'Erica Ford', 'Hannah Khalique-Brown', 'Mette', 'Marisa Abela', 'Lucy Boynton', 'Rob Brydon', 'Tom Stourton', 'Chris Tayl

In [11]:
# Assuming movie_data_dict is your dictionary
def insert_first_10000(movie_data_dict):
    """Inserts the first 10000 items from the dictionary."""

    first_10000 = {}
    keys = list(movie_data_dict.keys())

    for i in range(min(10000, len(keys))): #prevent error if the dictionary has less than 100 items
        print(keys[i])
        first_10000[keys[i]] = movie_data_dict[keys[i]]
        
    insert_movies_to_chroma(first_10000)

In [12]:
insert_first_10000(movie_data_dict)

1000001
1000002
1000003
1000004
1000005
1000006
1000007
1000008
1000009
1000010
1000011
1000012
1000013
1000014
1000015
1000016
1000017
1000018
1000019
1000020
1000021
1000022
1000023
1000024
1000025
1000026
1000027
1000028
1000029
1000030
1000031
1000032
1000033
1000034
1000035
1000036
1000037
1000038
1000039
1000040
1000041
1000042
1000043
1000044
1000045
1000046
1000047
1000048
1000049
1000050
1000051
1000052
1000053
1000054
1000055
1000056
1000057
1000058
1000059
1000060
1000061
1000062
1000063
1000064
1000065
1000066
1000067
1000068
1000069
1000070
1000071
1000072
1000073
1000074
1000075
1000076
1000077
1000078
1000079
1000080
1000081
1000082
1000083
1000084
1000085
1000086
1000087
1000088
1000089
1000090
1000091
1000092
1000093
1000094
1000095
1000096
1000097
1000098
1000099
1000100
1000101
1000102
1000103
1000104
1000105
1000106
1000107
1000108
1000109
1000110
1000111
1000112
1000113
1000114
1000115
1000116
1000117
1000118
1000119
1000120
1000121
1000122
1000123
1000124
1000125


In [14]:
def query_chroma_and_print_first(query_text, filter_criteria=None):
    
    try:
        if filter_criteria:
            results = collection.query(
                query_texts=[query_text],
                n_results=1,  # Get only the closest result
                where=filter_criteria,
            )
        else:
            results = collection.query(
                query_texts=[query_text],
                n_results=1,  # Get only the closest result
            )

        if results["documents"] and results["documents"][0]:
            print("Closest Result:")
            print("Document:", results["documents"][0][0])
            if results["metadatas"] and results["metadatas"][0]:
                print("Metadata:", results["metadatas"][0][0]) 
            else:
                print("No metadata found.")

        else:
            print("No results found.")

    except Exception as e:
        print(f"An error occurred: {e}")



In [13]:
print("Sample Documents:")
print(collection.peek())

Sample Documents:
{'ids': ['1000001', '1000002', '1000003', '1000004', '1000005', '1000006', '1000007', '1000008', '1000009', '1000010'], 'embeddings': array([[ 0.06744161,  0.03070172,  0.01605352, ..., -0.02811226,
        -0.00429987,  0.04156384],
       [-0.01754865,  0.05265442,  0.0240614 , ..., -0.03151114,
        -0.06095005,  0.05524215],
       [ 0.07866405,  0.03946168,  0.02145139, ...,  0.03446234,
        -0.00230261,  0.01832009],
       ...,
       [ 0.06645713, -0.03122009, -0.07642012, ...,  0.03748471,
         0.02360372, -0.03571105],
       [-0.009616  , -0.01523673, -0.07997236, ..., -0.08871879,
        -0.01516103,  0.00498322],
       [-0.01855909, -0.07260381, -0.05539532, ..., -0.06184743,
         0.11707151, -0.0170704 ]]), 'documents': ['Barbie - Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of l

In [29]:
# Example usage:
query_chroma_and_print_first("movies about friendships")



Closest Result:
Document: Scott Pilgrim vs. the World - Scott Pilgrim is a 22 year old radical Canadian wannabe rockstar who falls in love with an American delivery girl, Ramona Flowers, and must defeat her seven evil exes to be able to date her.
Metadata: {'actors': "Michael Cera Mary Elizabeth Winstead Ellen Wong Kieran Culkin Alison Pill Mark Webber Johnny Simmons Jason Schwartzman Anna Kendrick Aubrey Plaza Satya Bhabha Chris Evans Brandon Routh Mae Whitman Brie Larson Keita Saito Shota Saito Will Bowes Celine Lepage Mark Leroy Kjartan Hewitt Ben Lewis Nelson Franklin Kristina Pesic Matt Watts Ingrid Haas Maurie W. Kaufmann Marley Otto Christine Watson Chantelle Chung Don McKellar Jung-Yul Kim Erik Knudsen Abigail Chu Tennessee Thomas Emily Kassie Michael Lazarovitch John Patrick Amedori Joe Dinicol Craig Stickland Bill Hader Thomas Jane Clifton Collins Jr. Bryan Lee O'Malley", 'countries': 'Japan UK USA', 'crew': "Edgar Wright Joe Nozemack Lisa Gitter Steven V. Scavelli Eric Gitte

In [42]:
# Example with metadata filter (replace with your actual metadata fields):
query_chroma_and_print_first(
    "comedy movies with high ratings", filter_criteria={ "rating": {"$gt": 4.5}}
)



Closest Result:
Document: The Shawshank Redemption - Imprisoned in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.
Metadata: {'actors': 'Tim Robbins Morgan Freeman Bob Gunton William Sadler Clancy Brown Gil Bellows James Whitmore Mark Rolston Jeffrey DeMunn Larry Brandenburg Neil Giuntoli Brian Libby David Proval Joseph Ragno Jude Ciccolella Paul McCrane Renee Blaine Scott Mann John Horton Gordon Greene Alfonso Freeman Vincent Foster John E. Summers Frank Medrano Mack Miles Alan R. Kessler Morgan Lund Cornell Wallace Gary Lee Davis Neil Summers Ned Bellamy Joe Pecoraro Harold E. Cope Jr. Brian Delate Don McManus Donald Zinn Dorothy Silver Robert Haley Dana Snyd

In [46]:
#Example with an actor filter.
query_chroma_and_print_first(
    "movies starring Margot Robbie and ryan gosling called barbie"
)

Closest Result:
Document: Barbie - Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of living among humans.
Metadata: {'actors': 'Margot Robbie Ryan Gosling America Ferrera Ariana Greenblatt Issa Rae Kate McKinnon Alexandra Shipp Emma Mackey Hari Nef Sharon Rooney Ana Cruz Kayne Ritu Arya Dua Lipa Nicola Coughlan Emerald Fennell Simu Liu Kingsley Ben-Adir Ncuti Gatwa Scott Evans John Cena Michael Cera Rhea Perlman Helen Mirren Will Ferrell Connor Swindells Jamie Demetriou Andrew Leung Will Merrick Zheng Xi Yong Asim Chaudhry Ray Fearon Erica Ford Hannah Khalique-Brown Mette Marisa Abela Lucy Boynton Rob Brydon Tom Stourton Chris Taylor David Mumeni Olivia Brody Isla Ashworth Eire Farrell Daisy Duczmal Genvieve Toussaint Isabella Nightingale-Mercado Millie-Rose Crossley Anvita Nehru Kayla-Mai Alvares Luke Mullen Patrick Luwis Mac Br